In [4]:
pip install H2O


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for H2O: filename=h2o-3.42.0.3-py2.py3-none-any.whl size=240945068 sha256=fe18f4c86480f87b51aa369dbd1b58ca6e7ff92b6614c07ec38bc1247a259f16
  Stored in directory: /root/.cache/pip/wheels/f9/7d/1d/e1258f0558603969a8393cb5b22a18e313337f01627a118302
Successfully built H2O


In [22]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
from matplotlib import style
from tabulate import tabulate
import multiprocessing

In [6]:
plt.rcParams['image.cmap'] = "bwr"
plt.rcParams['figure.dpi'] = "100"
plt.rcParams['savefig.bbox'] = "tight"
style.use('ggplot') or plt.style.use('ggplot')

In [7]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [8]:
# Lee el archivo Excel en un DataFrame
df = pd.read_excel('/content/futbolargentino.xlsx')

# Ruta donde deseas guardar el archivo CSV resultante
archivo_csv = 'tu_archivo.csv'

# Guarda el DataFrame como un archivo CSV
df.to_csv(archivo_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV


In [11]:
pd.set_option('display.max_columns', None)
df.head()

,Jugadores,Posicion,Edad,Altura,Pie,Fichado,Equipo Anterior,Valor de mercado,Temporada,Club
0,Nicolás Gianni,Centrocampista,25,1.75,derecho,01/07/2000,Inferiores Argentinos Juniors,800000,2008,Argentinos Juniors
1,Maximiliano Sola,Defensa,22,1.8,izquierdo,-,NaN,-,2008,Argentinos Juniors
2,Nicolás Navarro,Portero,23,1.9,derecho,24/07/2010,SSC Nápoles,2000000,2008,Argentinos Juniors
3,Juan Carrera,Portero,27,1.91,derecho,01/01/2005,inactivo,800000,2008,Argentinos Juniors
4,Sebastián Torrico,Portero,28,1.86,derecho,01/01/2009,CD Godoy Cruz Antonio Tomba,250000,2008,Argentinos Juniors


In [17]:
# Obtén el número total de columnas
num_columnas = df.shape[1]

# Obtiene los nombres de todas las columnas
nombres_columnas = df.columns.tolist()

# Imprime el número total de columnas y sus nombres
print(f"Número total de columnas: {num_columnas}")
print(f"Nombres de las columnas: {nombres_columnas}")
df.columns.get_loc('Valor de mercado')


Número total de columnas: 10
Nombres de las columnas: ['Jugadores', 'Posicion', 'Edad', 'Altura', 'Pie', 'Fichado', 'Equipo Anterior', 'Valor de mercado', 'Temporada', 'Club']


7

In [13]:
# Creación de un cluster local H2O
# ------------------------------------------------------------------------------
h2o.init(ip = "localhost",
         # -1 indica que se empleen todos los cores disponibles.
         nthreads = -1,
         # Máxima memoria disponible para el cluster.
         max_mem_size = "4g")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.20.1" 2023-08-24; OpenJDK Runtime Environment (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.20.1+1-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpz6oltp71
  JVM stdout: /tmp/tmpz6oltp71/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpz6oltp71/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month and 7 days
H2O_cluster_name:,H2O_from_python_unknownUser_hwh6al
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [14]:
# Se eliminan los datos del cluster por si ya había sido iniciado.
h2o.remove_all()

In [15]:
# Carga tu conjunto de datos en H2O
datos_h2o = h2o.import_file('/content/tu_archivo.csv' , sep    = ",")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
# Dimensiones del set de datos
datos_h2o.shape

(12092, 10)

In [18]:
# Define las columnas de características y la columna objetivo
x = datos_h2o.columns[:6] + datos_h2o.columns[10:]  # Todas las columnas excepto la columna 7
y = datos_h2o.columns[7]   # La columna 7 (objetivo)

# Divide los datos en conjuntos de entrenamiento y prueba
train, test = datos_h2o.split_frame(ratios=[0.7], destination_frames= ["train", "test"], seed=123)

In [19]:
print(train['Valor de mercado'].table())
print(test['Valor de mercado'].table())

  Valor de mercado    Counts
                 0      1180
             10000         3
             15000         1
             25000       147
             45000         2
             50000       437
             60000         4
             75000       190
             80000         3
            100000       268
[150 rows x 2 columns]

  Valor de mercado    Counts
                 0       524
             25000        68
             45000         6
             50000       182
             75000        88
             80000         3
            100000        97
            125000        17
            150000        84
            175000        17
[118 rows x 2 columns]



In [ ]:
# En porcentaje
print(train["Total_2022"].table()["Count"]/train.shape[0])
print(test["Total_2022"].table()["Count"]/test.shape[0])

     Count
0.00152016
0.0538324
0.0351771
0.0199488
0.0129214
0.0201888
0.0107611
0.00958769
0.0106411
0.006294
[1317 rows x 1 column]

     Count
0.00205787
0.0548453
0.0356074
0.0195809
0.0135632
0.0201422
0.010757
0.00901098
0.0110377
0.00661013
[1317 rows x 1 column]



In [20]:
# Se comprueba que la variable respuesta es de tipo factor.
train['Valor de mercado'] = train['Valor de mercado'].asfactor()
test['Valor de mercado']  = test['Valor de mercado'].asfactor()
print(train['Valor de mercado'].isfactor())
print(train['Valor de mercado'].isfactor())

[True]
[True]


In [ ]:
# Inicializa AutoML
aml = H2OAutoML(max_models=3, seed=42)  # Puedes ajustar los parámetros según tus necesidades

# Entrena los modelos de AutoML
aml.train(x=x, y=y, training_frame=train)

# Verifica los modelos generados por AutoML
lb = aml.leaderboard
print(lb)

# Evalúa el mejor modelo en el conjunto de prueba
best_model = aml.leader
test_performance = best_model.model_performance(test_data=test)
print(test_performance)

# Puedes usar el modelo seleccionado para hacer predicciones en nuevos datos
# new_data = h2o.import_file("nuevos_datos.csv")
# predictions = best_model.predict(new_data)

# Detén el servidor H2O
h2o.shutdown()


AutoML progress: |█████████████████████████████████████████████████████████████